# Gợi ý các tin bài liên quan dựa trên sự tương đồng trong nội dung của bài viết

In [300]:
#library
import os
import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import re
from nltk.tokenize import word_tokenize
import codecs
import multiprocessing as mp
from random import shuffle
from bs4 import BeautifulSoup  
import requests
import pandas as pd
import readability
from readability.readability import Document
import urllib
from itertools import permutations
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

In [13]:
def string_clean(s):
    s_lower = s.lower()
    s_clean = " ".join(re.findall("[0-9a-zaáàảãạâấầẩẫậăắằẳẵặeéèẻẽẹêếềểễệiíìỉĩịoóòỏõọôốồổỗộơớờởỡợuúùủũụưứừửữựyýỳỷỹỵđ]+", s_lower))
    return(word_tokenize(s_clean))

# Đọc dữ liệu để train model
Dữ liệu của magizbox tại: https://github.com/magizbox/corpus.vinews/tree/master/vn_news/data. Dữ liệu của duyvuleo tại: https://github.com/duyvuleo/VNTC/tree/master/Data/27Topics/Ver1.1

In [85]:
news = list()
label = 0

#đọc dữ liệu của magizbox
folder_dir = '.../vietnam news/data'
file_list = os.listdir(folder_dir)

for i in file_list:
    file_dir = os.path.join(folder_dir, i)
    with open(file_dir, 'r') as f:
        label += 1
        news.append(gensim.models.doc2vec.TaggedDocument(string_clean(f.read()), ['news_' + str(label)]))
        
#đọc dữ liệu của duyvuleo, test data
#test
folder_dir = '../vietnam news/news test'
sub_folder_list = os.listdir(folder_dir)
for subs in sub_folder_list:
    file_list =  os.listdir(os.path.join(folder_dir, subs))
    for i in file_list:
        with open(os.path.join(folder_dir, subs,i), 'rb') as f:
            label += 1
            news.append(gensim.models.doc2vec.TaggedDocument(string_clean(f.read().decode("utf-16")), ['news_' + str(label)]))

#đọc dữ liệu của duyvuleo, train data
#test
folder_dir = '.../vietnam news/news train'
sub_folder_list = os.listdir(folder_dir)
for subs in sub_folder_list:
    file_list =  os.listdir(os.path.join(folder_dir, subs))
    for i in file_list:
        with open(os.path.join(folder_dir, subs,i), 'rb') as f:
            label += 1
            news.append(gensim.models.doc2vec.TaggedDocument(string_clean(f.read().decode("utf-16")), ['news_' + str(label)]))

# Xem qua dữ liệu

In [86]:
len(news)

61891

In [88]:
news[0:2]

[TaggedDocument(words=['kết', 'quả', 'xổ', 'số', 'điện', 'toán', 'vietlott', 'ngày', '6', '2', '2017', 'kết', 'quả', 'xổ', 'số', 'điện', 'toán', 'vietlott', 'ngày', '6', '2', '2017', 'kết', 'quả', 'xổ', 'số', 'điện', 'toán', '123', 'là', '2', '26', '733', 'xổ', 'số', 'thần', 'tài', '4', 'là', '3781', 'xổ', 'số', 'điện', 'toán', '6x36', 'là', '07', '08', '10', '12', '15', '16', 'kết', 'quả', 'xổ', 'sổ', 'điện', 'toán', 'vietlott', 'ngày', '6', '2', '2017', 'xổ', 'số', 'điện', 'toán', 'là', 'loại', 'hình', 'xổ', 'số', 'mà', 'trong', 'đó', 'người', 'chơi', 'được', 'phép', 'chọn', 'số', 'tùy', 'thích', 'có', 'thêm', 'nhiều', 'lựa', 'chọn', 'mua', 'vé', 'qua', 'internet', 'hay', 'thiết', 'bị', 'di', 'động', 'như', 'với', 'xổ', 'số', 'truyền', 'thống', 'đặc', 'biệt', 'là', 'giải', 'thưởng', 'của', 'xổ', 'số', 'điện', 'toán', 'sẽ', 'được', 'cộng', 'dồn', 'cho', 'đến', 'khi', 'có', 'người', 'nhận', 'thưởng', 'giúp', 'gia', 'tăng', 'giá', 'trị', 'thưởng', 'của', 'người', 'dùng', 'xổ', 'số', 'đi

# Tạo từ điển và train model

In [90]:
#build vocab and model
cores = mp.cpu_count()
model = Doc2Vec(min_count=1, window=15, size=100, workers=cores, alpha=0.025, min_alpha=0.025)
model.build_vocab(news)

In [94]:
#train model
#phần này sẽ tốn khá nhiều thời gian
for epoch in range(20):
    shuffle(news)
    model.train(news, total_examples=model.corpus_count, epochs=epoch)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

630407254

# Thu thập tin tức trên web

In [145]:
def clean_link(link):
    return(link.replace('\n','').strip())    

In [312]:
#thu thập dữ liệu trên web
def news_crawl():
    
    news_update = pd.DataFrame(columns = ['news_content', 'link'])
    
    list_rss = ['https://laodong.vn/rss/xa-hoi.rss',
                'https://laodong.vn/rss/thoi-su.rss',                
                'https://thanhnien.vn/rss/chinh-tri-xa-hoi.rss',
                'http://tuoitre.vn/rss/thoi-su.rss',
                'http://vietnamnet.vn/rss/thoi-su.rss',
                'http://dantri.com.vn/xa-hoi.rss',
                'https://vnexpress.net/rss/thoi-su.rss']
    
    for rss_link in list_rss:
        print('get news from ', rss_link)
        soup = BeautifulSoup(requests.get(rss_link).text, 'html.parser')
        links = [clean_link(link.get_text()) for link in [item.find('link') for item in soup.find_all('item')]]
        for link in links:
            html = requests.get(link).text
            readable_article = BeautifulSoup(Document(html).summary(), 'html.parser').get_text()
            readable_title = BeautifulSoup(Document(html).short_title(),'html.parser').get_text()
            news_update = news_update.append(pd.DataFrame({'news_content': [readable_title + ' ' + readable_article],
                                                          'link': [link]}))
    return(news_update)

In [313]:
%%time
list_news = news_crawl()

get news from  https://laodong.vn/rss/xa-hoi.rss
get news from  https://laodong.vn/rss/thoi-su.rss
get news from  https://thanhnien.vn/rss/chinh-tri-xa-hoi.rss
get news from  http://tuoitre.vn/rss/thoi-su.rss
get news from  http://vietnamnet.vn/rss/thoi-su.rss
get news from  http://dantri.com.vn/xa-hoi.rss
get news from  https://vnexpress.net/rss/thoi-su.rss
CPU times: user 17.8 s, sys: 312 ms, total: 18.1 s
Wall time: 9min 38s


In [314]:
#reset index
list_news = list_news.reset_index(drop=True)
list_news.head(10)

,link,news_content
0,https://laodong.vn/xa-hoi/xe-oto-cua-so-gddt-q...,"Xe ôtô của Sở GD&ĐT Quảng Bình gặp tai nạn, mộ..."
1,https://laodong.vn/xa-hoi/du-bao-mua-dong-nam-...,Dự báo mùa đông năm 2017: Miền Bắc rét đậm và ...
2,https://laodong.vn/xa-hoi/bo-nhiem-nhan-su-ban...,"Bổ nhiệm nhân sự Ban Tuyên giáo Trung ương, Bộ..."
3,https://laodong.vn/xa-hoi/giat-minh-voi-thong-...,“Giật mình” với thông tin áp thấp mới trên Biể...
4,https://laodong.vn/xa-hoi/chot-phuong-an-nghi-...,Chốt phương án nghỉ Tết Nguyên đán Mậu Tuất 7 ...
5,https://laodong.vn/xa-hoi/truong-dai-hoc-hutec...,Trường Đại học Hutech nói gì về sự cố sinh viê...
6,https://laodong.vn/xa-hoi/xoa-so-hang-loat-mai...,Xóa sổ hàng loạt mái che khu chợ Tam Bạc (Hải ...
7,https://laodong.vn/xa-hoi/bau-ban-chap-hanh-do...,Bầu Ban Chấp hành Đoàn TNCS Hồ Chí Minh TP Hà ...
8,https://laodong.vn/xa-hoi/cho-thoi-chuc-giam-d...,Cho thôi chức Giám đốc Sở KH&CN Ninh Bình đối ...
9,https://laodong.vn/xa-hoi/thu-ngan-sach-cham-p...,"Thu ngân sách chậm, phải ứng tiền tỉ trả lương..."


# Vectorize

In [315]:
#từ điển
word_vectors = model.wv.vocab
ranges = len(list_news)
list_news_vec = np.zeros((ranges, 100))

for link, news_content, index in zip(list_news['link'], list_news['news_content'], range(ranges)):
    #tách từ và chỉ lấy các từ có trong từ điển
    news_clean = [x for x in string_clean(news_content) if x in word_vectors]
    #chuyển thành vector
    list_news_vec[index] = model.infer_vector(news_clean)

In [316]:
list_news_vec

array([[ 1.21588886,  0.25520918,  0.1180691 , ..., -1.23956525,
         0.42185211, -0.49556363],
       [ 0.54214287,  0.25655559, -0.31910422, ..., -1.07144845,
         0.45198804, -1.70662868],
       [ 2.84379101,  0.34197253, -0.59917337, ..., -0.16288888,
        -1.17410326,  0.26831767],
       ..., 
       [ 1.06142628,  0.99939489,  0.30169684, ..., -0.43171528,
         0.49284247,  0.59281617],
       [ 0.20131224,  0.38381097,  0.18135981, ..., -0.3422454 ,
         0.91234881, -0.41544929],
       [ 0.19560772,  0.20024474, -0.22636834, ..., -0.46932459,
         0.78941023, -0.41454673]])

# Tìm các tin liên quan bằng K-NN

In [512]:
#Sử dụng 10-NN để tìm 10 đối tượng gần nhất
neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(list_news_vec)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

# Áp dụng để gợi ý các tin liên quan

In [532]:
def news_recommend(text):
    (distance, found_index) = neigh.kneighbors(model.infer_vector(string_clean(news_text)).reshape(1, -1))
    print('Các tin liên quan:', '\n')
    for index in found_index.tolist()[0]:
        print(list_news.link[index])
        print(re.sub( '\s+', ' ',list_news.news_content[index][0:100].replace('\n', '')), '\n')        

In [534]:
news_text = """
Nam sinh tử vong tại sân trường vì bêtông rơi trúng đầu
Đang xếp hàng chờ thang máy tại Đại học Hutech, TP HCM, nam sinh bất ngờ bị mảng bêtông rơi từ trên cao trúng đầu tử vong.
nam-sinh-tu-vong-tai-san-truong-vi-betong-roi-trung-dau

Hiện trường vụ việc. Ảnh: Cộng đồng sinh viên Hutech

Chiều 17/10, hơn chục sinh viên xếp hàng đợi thang máy tại khu B Đại học Công nghệ TP HCM (Hutech), đường Điện Biên Phủ, quận Bình Thạnh. Bất ngờ, một mảng bêtông từ trên cao rơi trúng đầu một nam sinh khiến người này gục xuống sân trường. Hàng chục sinh viên chứng kiến vụ việc la hét tháo chạy.

Tại hiện trường, nạn nhân mặc áo sơ mi trắng, tử vong trong tư thế úp mặt xuống đất. Mảnh bêtông vỡ nát, nằm vương vãi xung quanh. 

"Mình đang đứng xếp hàng thì nghe tiếng động lớn phía sau, nhiều mảnh bêtông nhỏ văng trúng người nhưng không sao. Quay lại, mình thấy một bạn nam nằm gục dưới sàn với vũng máu", một nữ sinh kể.
nam-sinh-tu-vong-tai-san-truong-vi-betong-roi-trung-dau-1

Hiện trường vụ việc từ trên cao. Ảnh: Cộng đồng sinh viên Hutech

Theo đại diện Hutech, nam sinh tên Nguyễn Thanh Long (29 tuổi, ở TP HCM), sinh viên khoa Kỹ thuật Môi trường. Nguyên nhân vụ việc đang được trường phối hợp với cảnh sát làm rõ.

"""

news_recommend(news_text)

Các tin liên quan: 

https://vnexpress.net/tin-tuc/giao-duc/nam-sinh-tu-vong-tai-san-truong-vi-betong-roi-trung-dau-3657229.html
Nam sinh tử vong tại sân trường vì bêtông rơi trúng đầu Hiện trường vụ việc. Ảnh: Cộng đồn 

https://vnexpress.net/tin-tuc/giao-duc/nam-sinh-dai-hoc-hutech-tu-vong-vi-manh-vo-cua-mang-xoi-nuoc-3657501.html
Nam sinh Đại học HUTECH tử vong vì mảnh vỡ của máng xối nước Hiện trường vụ việc nhìn từ trên c 

https://video.vnexpress.net/tin-tuc/xa-hoi/nam-thanh-nien-lao-xuong-song-da-nhim-cuu-hai-nu-sinh-3657727.html
Nam thanh niên lao xuống sông Đa Nhim cứu hai nữ sinh Nam thanh niên lao xuống sông Đa Nhim cứu ha 

https://video.vnexpress.net/tin-tuc/xa-hoi/xe-may-nga-nhao-lien-tuc-khi-len-loi-tranh-duong-tac-o-ha-noi-3657247.html
Xe máy ngã nhào liên tục khi len lỏi tránh đường tắc ở Hà Nội Xe máy ngã nhào liên tục khi len lỏi 

http://vietnamnet.vn/vn/thoi-su/an-toan-giao-thong/tren-duong-di-hoc-nam-sinh-lop-9-bi-xe-tai-can-tu-vong-405382.html
Trên đường đi học, 

In [535]:
news_text = """
Hà Nội bắt đầu di dời 1.200 cây làm đường vành đai 3
Việc đánh chuyển, chặt hạ số cây xanh dự kiến kéo dài đến trước tháng 12 năm nay.

14 cây xanh sẽ bị chặt hoặc đánh chuyển trong đợt đầu, để bàn giao mặt bằng cho dự án mở rộng đường vành đai 3 đoạn Mai Dịch - cầu Thăng Long và dự án đường trên cao.

Theo khảo sát, có 1.159 cây xanh trên đường Phạm Văn Đồng phải di dời. Với những cây có thể trồng được, nhà chức trách sẽ đưa về nút giao Tả Hồng - Võ Nguyên Giáp và nút giao quốc lộ 5 - vành đai 3.

Chủ tịch Hà Nội cho rằng cây xà cừ đã đánh chuyển không thể trồng lại trên phố. 

Số tiền để đánh chuyển, ươm trồng 1.000 cây trên đường Phạm Văn Đồng có thể dùng để trồng mới hàng chục nghìn cây khác.

Mỗi cây đều có mã số riêng để nhận biết.

Việc di chuyển cây xanh trên đường Phạm Văn Đồng dự kiến triển khai từ giữa năm 2017. Tuy nhiên, khi chủ đầu tư công bố kế hoạch, có một số ý kiến chưa đồng tình.

Sau thời gian tạm dừng, việc chặt hạ tiếp tục tiến hành như kế hoạch ban đầu và dự kiến hoàn thành trước tháng 12 năm nay.

Việc cắt tỉa và di dời cây vào giờ cao điểm sáng làm tắc cục bộ trên đường Phạm Văn Đồng.
"""

news_recommend(news_text)

Các tin liên quan: 

https://video.vnexpress.net/tin-tuc/xa-hoi/ha-noi-danh-chuyen-gan-1-200-cay-de-lam-duong-vanh-dai-3-3657491.html
Hà Nội đánh chuyển gần 1.200 cây để làm đường vành đai 3 Hà Nội đánh chuyển gần 1.200 cây để làm đ 

https://vnexpress.net/photo/thoi-su/ha-noi-bat-dau-di-doi-1-200-cay-lam-duong-vanh-dai-3-3657460.html
Hà Nội bắt đầu di dời 1.200 cây làm đường vành đai 3 Theo khảo sát, có 1.159 cây xanh trên đường P 

https://video.vnexpress.net/tin-tuc/xa-hoi/xe-may-nga-nhao-lien-tuc-khi-len-loi-tranh-duong-tac-o-ha-noi-3657247.html
Xe máy ngã nhào liên tục khi len lỏi tránh đường tắc ở Hà Nội Xe máy ngã nhào liên tục khi len lỏi 

https://laodong.vn/media/40-nam-gia-nhap-lhq-viet-nam-ngay-cang-khang-dinh-vi-the-cua-minh-570518.ldo
40 năm gia nhập LHQ, Việt Nam ngày càng khẳng định vị thế của mình Trong 40 năm gia nhập LHQ, vị th 

https://vnexpress.net/tin-tuc/giao-duc/nam-sinh-dai-hoc-hutech-tu-vong-vi-manh-vo-cua-mang-xoi-nuoc-3657501.html
Nam sinh Đại học HUTECH tử

In [536]:
news_text = """
Hà Văn Thắm gửi đơn kháng cáo viết tay dài 9 trang giấy | Thời sự

Trong đơn kháng cáo, Hà Văn Thắm xin chấp hành bản án tòa cấp sơ thẩm tuyên ngày 29.9 về 2 tội danh "Vi phạm quy định về cho vay trong hoạt động của các tổ chức tín dụng" và "Cố ý làm trái các quy định của Nhà nước về quản lý kinh tế gây hậu quả nghiêm trọng".

Nội dung đơn kháng cáo của nguyên Chủ tịch HĐQT OceanBank

 cũng bày tỏ nguyện vọng tòa cấp cao xem xét lại phần tuyên phạt mình 20 năm tù về tội "Lạm dụng chức vụ chiếm đoạt tài sản" và mức án tù chung thân về tội "Tham ô tài sản".

Ngoài ra, Hà Văn Thắm trình bày nếu tòa cấp cao thấy đủ căn cứ kết tội về 2 tội danh trên, thì Hà Văn Thắm xin được tòa cấp cao xem xét 5 tình tiết giảm nhẹ, như: Không được hưởng lợi; hành vi chi tiền vượt trần lãi suất được thực hiện trong hoàn cảnh bắt buộc, hơn nữa bản thân luôn thành khẩn khai báo...

Với phán quyết của tòa sơ thẩm về trách nhiệm dân sự, Hà Văn Thắm cho rằng việc bản thân phải bồi thường hơn 800 tỉ đồng cho OceanBank (mới) là chưa đúng, và khẳng định không có khả năng bồi thường thiệt hại như án sơ thẩm đã tuyên.

Liên quan đến số tiền 197 tỉ đồng mà bị cáo Nguyễn Xuân Sơn đã chiếm đoạt của OceanBank, Hà Văn Thắm cho rằng tòa sơ thẩm tuyên bị cáo Sơn phải bồi thường thiệt hại cho PVN 49 tỉ đồng (tương đương 20% cổ phần của PVN tại OceanBank), vậy với tư cách là cổ đông của OceanBank, Hà Văn Thắm cũng phải được bồi thường tương ứng với số cổ phần 62,9% mà bị cáo này nắm giữ.

Cùng ngày, Tòa án nhân dân TP.Hà Nội cũng cho biết, tới thời điểm hiện tại, nếu tính cả trường hợp của Hà Văn Thắm, cơ quan này đã nhận được tổng cộng 21 đơn kháng cáo

, trong tổng số 51 bị cáo bị đưa ra xét xử sơ thẩm trong đại án OceanBank mới đây.


"""

news_recommend(news_text)

Các tin liên quan: 

https://thanhnien.vn/thoi-su/ha-van-tham-gui-don-khang-cao-viet-tay-dai-9-trang-giay-891386.html
Hà Văn Thắm gửi đơn kháng cáo viết tay dài 9 trang giấy Trong đơn kháng cáo, Hà Văn Thắm xin chấp h 

https://thanhnien.vn/thoi-su/1851-bi-cao-vu-dai-an-oceanbank-gui-don-khang-cao-891209.html
18/51 bị cáo vụ 'đại án OceanBank' gửi đơn kháng cáo Ngày 17.10, thẩm phán Trần Nam Hà, chủ tọa phi 

https://vnexpress.net/photo/thoi-su/sau-cong-trinh-kien-truc-phap-tram-tuoi-tai-ha-noi-xua-va-nay-3653630.html
Sáu công trình kiến trúc Pháp trăm tuổi tại Hà Nội xưa và nay Cầu Doumer (cầu Long Biên ngày nay)  

https://thanhnien.vn/thoi-su/tam-giam-doi-nam-nu-mua-ban-hon-3-kg-co-my-891119.html
Tạm giam đôi nam nữ mua bán hơn 3 kg 'cỏ Mỹ' Chiều 17.10, Cơ quan Cảnh sát điều tra Công an Q.Sơn T 

https://thanhnien.vn/thoi-su/hai-ke-gay-ra-nhieu-vu-cuop-giat-sa-luoi-891464.html
Hai kẻ gây ra nhiều vụ cướp giật sa lưới Theo công an, khoảng 17 giờ ngày 17.10 Duy điều khiển xe g 



In [537]:
news_text = """
Thời tiết 18/10: Miền Bắc hanh khô, miền Trung mưa to

- Từ hôm nay miền Bắc tăng nhiệt dần nhưng trời nắng hanh, trong khi nửa miền Trung kéo vào đến Nam Bộ có mưa vừa, mưa to đến rất to.

2 ngày qua, người dân miền Bắc đã hưởng trọn vẹn đợt se lạnh đầu mùa, vùng núi chuyển rét. Đến đêm qua, nhiệt độ bắt đầu nhích nhẹ, thủ đô Hà Nội thấp nhất 20 độ, TP Lạng Sơn lạnh 17 độ, TP Hải Phòng, Sơn La 18 độ, TP Hà Giang 19 độ, riêng Sa Pa (Lào Cai) lạnh 12 độ.

Sang đến hôm nay, mưa nhỏ chỉ còn diễn ra một số nơi tại miền Bắc, trưa chiều trời tạnh ráo, có lúc hửng nắng, nhiệt độ tăng khoảng 3 độ.
Dự báo thời tiết, bản tin thời tiết, tin thời tiết, thời tiết Hà Nội
Miền Bắc tạnh ráo nhưng sẽ nắng hanh trong nhiều ngày tới

Nhiệt độ cao nhất tại TP Lạng Sơn lên 22 độ, Hải Phòng, Sơn La, Hà Giang 24 độ, thủ đô Hà Nội tăng lên 26 độ, Sa Pa tăng lên 16-17 độ C, TP Điện Biên Phủ 28 độ...

Đà tăng nhiệt sẽ tiếp tục, ngày mai cao nhất tại Hà Nội sẽ lên mức 28-29 độ C, trời nắng ráo nhưng hanh khô, se lạnh về đêm và sáng sớm với nhiệt độ phổ biến 22-23 độ C.

Trong khi đó dọc khắp từ Quảng Bình vào đến Nam Bộ từ hôm nay đến hết ngày 19/10 sẽ có mưa diện rộng, do dải hội tụ nhiệt đới vắt ngang qua Nam Trung Bộ.

Cụ thể, các tỉnh từ Quảng Bình đến Quảng Ngãi mưa phổ biến từ 70-100mm/đợt, riêng Quảng Trị, Đà Nẵng, Quảng Nam 100-150mm/đợt, có nơi trên 200mm.

Các tỉnh từ Bình Định đến Bình Thuận có mưa phổ biến từ 50-100mm/đợt; Các tỉnh phía Nam Tây Nguyên (các tỉnh Đắk Lắk, Đắk Nông, Lâm Đồng) và khu vực Nam Bộ có mưa từ 70-150mm/đợt.
 

Do mưa lớn, trên các sông từ Quảng Bình đến Bình Thuận và khu vực nam Tây Nguyên sẽ xuất hiện 1 đợt lũ. Trong đợt lũ này, mực nước trên các sông phổ biến còn dưới mức BĐ1, riêng các sông ở Ninh Thuận, Bình Thuận có khả năng lên mức BĐ2-BĐ3.

Ngoài ra các tỉnh Bình Thuận, Ninh Thuận, Lâm Đồng cần lưu ý nguy cơ cao xảy ra lũ quét, sạt lở đất ở vùng núi, ngập úng cục bộ ở vùng trũng thấp.

Riêng Nam Bộ, do mưa lớn kết hợp với triều cường, nguy cơ cao xảy ra ngập lụt ở các vùng trũng thấp thuộc khu vực Nam Bộ, đặc biệt tại TP.HCM, Vĩnh Long, Cần Thơ.


"""

news_recommend(news_text)

Các tin liên quan: 

http://vietnamnet.vn/vn/thoi-su/thoi-tiet-18-10-mien-bac-hanh-kho-mien-trung-mua-to-405332.html
Miền Bắc hanh khô, miền Trung mưa to - Từ hôm nay miền Bắc tăng nhiệt dần nhưng trời nắng hanh, tron 

https://vnexpress.net/tin-tuc/thoi-su/bien-dong-xuat-hien-vung-ap-thap-nam-bo-mua-lon-3657630.html
Biển Đông xuất hiện vùng áp thấp, Nam Bộ mưa lớn 13h ngày 18/10, trung tâm vùng áp thấp cách đảo  

http://tuoitre.vn/bien-dong-xuat-hiep-ap-thap-mua-dong-tren-bien-lan-dat-lien-20171018151942411.htm
Biển Đông xuất hiệp áp thấp, mưa dông trên biển lẫn đất liền Sơ đồ dự báo hướng đi vùng áp thấp - Ả 

http://dantri.com.vn/xa-hoi/xuat-hien-vung-ap-thap-tren-bien-dong-20171018162224746.htm
Xuất hiện vùng áp thấp trên Biển Đông Vị trí và hướng di chuyển của vùng áp thấp nhiệt đới. (Ảnh 

https://laodong.vn/xa-hoi/giat-minh-voi-thong-tin-ap-thap-moi-tren-bien-dong-570799.ldo
“Giật mình” với thông tin áp thấp mới trên Biển Đông Chiều nay (18.10), trên khu vực phía Đông Bắc  

